In [1]:
import os, sys
sys.path.append("/home/files/feature_orthogonality")
from utils.session_config import setup_gpus

In [2]:
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


# Load mnist images

In [4]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
train_x = train_x[:,:,:,tf.newaxis]/255.
test_x = test_x[:,:,:,tf.newaxis]/255.
print(train_x.shape)
print(test_x.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


# Build Model

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization
from utils.losses import L2_Orthogonal

model = tf.keras.Sequential()
model.add(Conv2D(32, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(64, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(32, kernel_regularizer=L2_Orthogonal()))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dense(10, activation='softmax',kernel_regularizer=L2_Orthogonal()))

In [6]:
initial_model = tf.keras.models.load_model("cnn_initialized.h5")

In [7]:
initial_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                51232     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [8]:
model(tf.keras.Input(shape=(28,28,1)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 32)        128       
_________________________________________________________________
re_lu (ReLU)                 (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 64)        256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 11, 11, 64)        0

In [9]:
for layer in initial_model.layers:
    print(layer.name)
    model.get_layer(layer.name).set_weights(layer.get_weights())

conv2d
max_pooling2d
conv2d_1
max_pooling2d_1
flatten
dense
dense_1


In [10]:
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])
model.fit(train_x, train_y, epochs=10)
model.evaluate(test_x, test_y)

Epoch 1/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1966 - acc: 0.9674
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1088 - acc: 0.9834
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0958 - acc: 0.9862
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0870 - acc: 0.9876
Epoch 5/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0806 - acc: 0.9886
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0726 - acc: 0.9905
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0699 - acc: 0.9901
Epoch 8/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0644 - acc: 0.9915
Epoch 9/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0619 - acc: 0.9915
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0.0645 - acc: 0.9909


[0.06446605920791626, 0.9908999800682068]

# Visualization

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def get_weight_cosine_similarity(weight):
    w = weight
    if len(w.shape) == 4: #convolution
        H,W,C,N = w.shape
        row_dims = H*W*C
        col_dims = N
    else: #dense
        D,N = w.shape
        row_dims = D
        col_dims = N
    w = tf.reshape(w, (row_dims, col_dims))
    norm = tf.norm(w, axis=0)
    w = w/norm #normalize
    wT = tf.transpose(w)
    correlations = tf.matmul(wT,w)
    return correlations

def get_weight_l2norm(weight):
    w = weight
    if len(w.shape) == 4: #convolution
        H,W,C,N = w.shape
        row_dims = H*W*C
        col_dims = N
    else: #dense
        D,N = w.shape
        row_dims = D
        col_dims = N
    w = tf.reshape(w, (row_dims, col_dims))
    norm = tf.norm(w, axis=0)
    return norm

In [ ]:
weight = model.get_layer("conv2d").get_weights()[0]
correlations = get_weight_cosine_similarity(weight)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 800
sns.heatmap(correlations, vmin=-1, vmax=1, cmap='RdBu_r', center=0, annot=True, fmt='.1f',xticklabels=False, yticklabels=False,annot_kws={"size": 4})

In [ ]:
norm = get_weight_l2norm(weight)
plt.bar(range(correlations.shape[0]), norm)
plt.title('Norm of weights', fontsize=20)
plt.show()

In [ ]:
model.save("orthogonal_cnn.h5")